In [1]:
pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00


In [2]:
pip install wolframalpha

In [3]:
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
import json
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import wolframalpha
import logging
import asyncio
import nest_asyncio
from google.colab import drive

In [4]:
# Configuración inicial para Colab
nest_asyncio.apply()
drive.mount('/content/drive')

# Configuración de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

Mounted at /content/drive


In [5]:
class CerebroCalculusColab:
    def __init__(self, db_path="/content/drive/MyDrive/Proyecto_Math/calculus_db.json"):
        self.db_path = Path(db_path)
        self.bd = self.cargar_bd()
        self.constantes = self.bd.get("constantes", {})
        self.historial = []
        self.cache = {}
        self.entrenar_modelo()
        self.wolfram_client = None
        self.configurar_wolfram()

    def cargar_bd(self):
        """Carga la base de datos con manejo robusto de errores"""
        try:
            with open(self.db_path, 'r', encoding='utf-8') as f:
                bd = json.load(f)
                if "derivadas" not in bd or "ejemplos_entrenamiento" not in bd:
                    raise ValueError("Estructura de BD inválida")
                return bd
        except Exception as e:
            logger.error(f"Error cargando BD: {str(e)}")
            return {
                "constantes": {"π": 3.1416, "e": 2.7183},
                "derivadas": {"tabla_completa": {}},
                "ejemplos_entrenamiento": []
            }

    def entrenar_modelo(self):
        """Entrena el clasificador de problemas matemáticos"""
        ejemplos = self.bd.get("ejemplos_entrenamiento", [])
        if len(ejemplos) < 3:
            logger.warning("Insuficientes datos de entrenamiento. Usando clasificador básico.")
            self.modelo = None
            return

        try:
            vectorizer = TfidfVectorizer()
            X = vectorizer.fit_transform([e["input"] for e in ejemplos])
            y = [e["tipo"] for e in ejemplos]

            self.modelo = SVC(kernel='linear')
            self.modelo.fit(X, y)
            self.vectorizer = vectorizer
            logger.info(f"Modelo entrenado con {len(ejemplos)} ejemplos")
        except Exception as e:
            logger.error(f"Error entrenando modelo: {str(e)}")
            self.modelo = None

    def configurar_wolfram(self, api_key="9EW5A6-VTQKYPA9JH"):
        """Configura Wolfram Alpha"""
        try:
            self.wolfram_client = wolframalpha.Client(api_key)
            logger.info("Wolfram Alpha configurado")
        except Exception as e:
            logger.error(f"Error configurando Wolfram: {str(e)}")
            self.wolfram_client = None

    async def resolver_async(self, consulta: str):
        """Versión asíncrona del resolver con caché"""
        if consulta in self.cache:
            logger.info(f"Usando caché para: {consulta}")
            return self.cache[consulta]

        try:
            resultado = await asyncio.to_thread(self.resolver, consulta)
            self.historial.append({"consulta": consulta, "resultado": resultado})
            self.cache[consulta] = resultado
            return resultado
        except Exception as e:
            logger.error(f"Error procesando {consulta}: {str(e)}")
            raise HTTPException(status_code=400, detail=str(e))

    def resolver(self, consulta):
        """Resuelve problemas con fallback a Wolfram Alpha"""
        try:
            tipo = self.clasificar_consulta(consulta)

            if tipo == "derivada":
                resultado = self.resolver_derivada(consulta)
            elif tipo == "limite":
                resultado = self.resolver_limite(consulta)
            elif tipo == "integral":
                resultado = self.resolver_integral(consulta)
            else:
                raise ValueError("Tipo de operación no soportado")

            return {
                "problema": consulta,
                "solucion": str(resultado['solucion']),
                "metodo": resultado.get('metodo', 'SymPy'),
                "pasos": resultado.get('pasos', ["Proceso completado"]),
                "tipo": tipo
            }

        except Exception as e:
            if self.wolfram_client:
                try:
                    return self.resolver_con_wolfram(consulta)
                except Exception as wolfram_error:
                    raise ValueError(f"Error local y en Wolfram: {str(wolfram_error)}")
            raise ValueError(f"Error: {str(e)}")

    def resolver_derivada(self, consulta):
        """Resuelve derivadas con BD local o SymPy"""
        try:
            expr_str = consulta.split("derivar")[1].strip()
            expr_limpia = self.preprocesar_expresion(expr_str)

            # Buscar en tabla de derivadas
            tabla = self.bd.get("derivadas", {}).get("tabla_completa", {})
            if expr_str in tabla:
                return {
                    "problema": consulta,
                    "solucion": tabla[expr_str],
                    "metodo": "BD (tabla)",
                    "pasos": [f"Derivada directa de {expr_str}"]
                }

            # Resolver con SymPy
            x = sp.symbols('x')
            expr = parse_expr(expr_limpia)
            derivada = sp.diff(expr, x)

            return {
                "problema": consulta,
                "solucion": str(derivada),
                "metodo": "SymPy",
                "pasos": [
                    f"1. Expresión: {expr}",
                    "2. Aplicando reglas de derivación...",
                    f"3. Resultado: {derivada}"
                ]
            }
        except Exception as e:
            raise ValueError(f"No se pudo resolver la derivada: {str(e)}")
    def resolver_con_wolfram(self, consulta):
        """Consulta a Wolfram Alpha"""
        try:
            res = self.wolfram_client.query(consulta)
            wolfram_result = next(res.results).text

            return {
                "problema": consulta,
                "solucion": wolfram_result,
                "metodo": "Wolfram Alpha",
                "pasos": ["Consulta externa a Wolfram Alpha"],
                "detalles_wolfram": str(res.details)  # Información adicional
            }
        except Exception as e:
            raise ValueError(f"Error consultando Wolfram Alpha: {str(e)}")
    def preprocesar_expresion(self, expr):
        """Formatea la expresión para SymPy"""
        # Reemplaza constantes conocidas
        for const, val in self.constantes.items():
            expr = expr.replace(const, str(val))

        # Reemplaza notación matemática
        expr = (expr.replace("^", "**")
                .replace("sen(", "sin(")
                .replace("π", "pi"))

        # Manejo de espacios en expresiones como "x 2" -> "x*2"
        expr = ''.join(c if c.isalnum() else f'*{c}' if c.isspace() else c for c in expr)

        return expr
    def resolver_limite(self, consulta):
        """Resuelve límites matemáticos"""
        try:
            # Extraer componentes de la consulta
            if "cuando" in consulta:
                expr_part, lim_part = consulta.split("cuando")
                expr_str = expr_part.replace("limite", "").strip()
                var, valor = lim_part.strip().split("->")
            else:
                expr_str = consulta.replace("limite", "").strip()
                var, valor = "x", "0"  # Valores por defecto

            # Procesar expresión
            expr_limpia = self.preprocesar_expresion(expr_str)
            x = sp.symbols(var.strip())
            punto = parse_expr(valor.strip())

            # Calcular límite
            limite = sp.limit(parse_expr(expr_limpia), x, punto)

            return {
                "problema": consulta,
                "solucion": str(limite),
                "metodo": "SymPy (límite)",
                "pasos": [
                    f"1. Expresión: {expr_limpia}",
                    f"2. Variable: {var.strip()} -> {valor.strip()}",
                    f"3. Resultado: {limite}"
                ]
            }
        except Exception as e:
            raise ValueError(f"No se pudo resolver el límite: {str(e)}")

In [6]:
import os
if not os.path.exists('static'):
    os.makedirs('static')

app = FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount("/static", StaticFiles(directory="static"), name="static")

# Inicialización del cerebro de cálculo
cerebro = CerebroCalculusColab()

@app.get("/", response_class=HTMLResponse)
async def interfaz_web(request: Request):
    return templates.TemplateResponse("index_colab.html", {
        "request": request,
        "results": [],
        "historial": cerebro.historial[-5:]
    })

@app.post("/api/calculate")
async def calcular(consulta: str):
    try:
        resultado = await cerebro.resolver_async(consulta)
        return JSONResponse(resultado)
    except HTTPException as e:
        return JSONResponse({"error": e.detail}, status_code=e.status_code)

In [7]:
# Configuración especial para Colab
def run_in_colab():
    from pyngrok import ngrok
    import uvicorn
    import threading

    # Configurar ngrok
    ngrok_tunnel = ngrok.connect(8501)
    print(f"Accede a la interfaz web en: {ngrok_tunnel.public_url}")

    # Iniciar servidor Uvicorn en segundo plano
    threading.Thread(target=uvicorn.run,
                    args=(app,),
                    kwargs={"host": "0.0.0.0", "port": 8501},
                    daemon=True).start()

# Crear estructura de archivos necesaria
!mkdir -p templates static
!pip install fastapi uvicorn pyngrok nest-asyncio sympy scikit-learn wolframalpha

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


In [8]:
!mkdir -p templates static

In [9]:
!pip install bootstrap-icons

ERROR: Could not find a version that satisfies the requirement bootstrap-icons (from versions: none)
ERROR: No matching distribution found for bootstrap-icons


In [10]:
%%writefile templates/index_colab.html
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="description" content="Sistema inteligente para resolver problemas matemáticos avanzados">

    <!-- Favicon (asegúrate de tener el archivo favicon.ico en /static) -->
    <link rel="icon" href="/static/favicon.ico" type="image/x-icon">

    <!-- Bootstrap Icons -->
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.10.0/font/bootstrap-icons.css">

    <!-- Tu hoja de estilos -->
    <link rel="stylesheet" href="/static/style.css">

    <title>Cerebro de Cálculo - Proyecto de Título</title>
</head>
<body>
    <div class="container">
        <header class="header">
            <h1><i class="bi bi-calculator"></i> Cerebro de Cálculo</h1>
            <p class="subtitle">Proyecto de Título - [Tu Nombre]</p>
        </header>

        <main class="main-content">
            <section class="input-section">
                <form id="calc-form" class="math-form">
                    <div class="form-group">
                        <label for="consulta" class="form-label">
                            <i class="bi bi-pencil-square"></i> Ingrese su problema matemático:
                        </label>
                        <textarea id="consulta" class="form-input" rows="3"
                                  placeholder="Ejemplos:&#10;• derivar x^2 + sin(x)&#10;• integrar 3x^2 + 2x&#10;• limite (x^2-1)/(x-1) cuando x->1"
                                  required></textarea>
                    </div>
                    <button type="submit" class="submit-btn">
                        <i class="bi bi-gear"></i> Resolver
                    </button>
                </form>
            </section>

            <section class="results-section">
                <h2 class="section-title"><i class="bi bi-graph-up"></i> Resultados</h2>
                <div id="result-container" class="result-container">
                    <!-- Los resultados aparecerán aquí dinámicamente -->
                    <div class="placeholder">
                        <i class="bi bi-arrow-up-circle"></i>
                        <p>Ingresa un problema matemático para ver la solución</p>
                    </div>
                </div>
            </section>
        </main>

        <footer class="footer">
            <p>Sistema desarrollado como proyecto de título - © 2023</p>
        </footer>
    </div>

    <!-- JavaScript -->
    <script src="/static/main.js"></script>
</body>
</html>

Writing templates/index_colab.html


In [11]:
if __name__ == "__main__":
    try:
        print("🚀 Iniciando Cerebro de Cálculo...")
        run_in_colab()

        # Mantener el notebook activo
        while True:
            pass

    except KeyboardInterrupt:
        print("\n🛑 Servidor detenido")
    except Exception as e:
        print(f"❌ Error: {str(e)}")

🚀 Iniciando Cerebro de Cálculo...


ERROR:pyngrok.process.ngrok:t=2025-05-15T22:40:56+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-15T22:40:56+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-15T22:40:56+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

❌ Error: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.


In [4]:
from IPython.display import display, Markdown, Math
import sympy as sp
from sympy.parsing.sympy_parser import (
    parse_expr,
    standard_transformations,
    implicit_multiplication_application
)
from datetime import datetime

transformations = (standard_transformations + (implicit_multiplication_application,))

class DerivadorAvanzado:
    def __init__(self, variable='x'):
        """
        Derivador avanzado con soporte para derivadas complejas y explicación detallada.

        Args:
            variable (str): Variable de derivación (por defecto 'x')
        """
        self.variable = sp.Symbol(variable)
        self.historial = []
        print(f"🚀 Derivador Avanzado inicializado (variable: '{variable}')")
        print("Métodos soportados:")
        print("- Reglas básicas (potencia, suma, producto, cociente)")
        print("- Funciones trigonométricas e hiperbólicas")
        print("- Funciones exponenciales y logarítmicas")
        print("- Derivación implícita")
        print("- Derivadas de orden superior")

    def calcular_derivada(self, expresion, orden=1):
        """
        Calcula derivadas con explicación paso a paso.

        Args:
            expresion (str): Expresión a derivar
            orden (int): Orden de derivación (1 para primera derivada, 2 para segunda, etc.)

        Returns:
            sympy.Expr: Resultado de la derivada
        """
        try:
            expr = self._parsear_expresion(expresion)
            display(Markdown(f"### 🔍 Derivando ({orden}° orden): `{expr}`"))

            if orden == 1:
                resultado = self._derivar_paso_a_paso(expr)
            else:
                resultado = self._derivar_orden_superior(expr, orden)

            self._agregar_al_historial(expr, resultado, orden)
            return resultado

        except Exception as e:
            display(Markdown(f"❌ **Error:** `{str(e)}`"))
            return None

    # ========== MÉTODOS PRINCIPALES DE DERIVACIÓN ==========
    def _derivar_paso_a_paso(self, expr):
        """Calcula la derivada mostrando cada paso con reglas aplicadas."""
        x = self.variable

        # Casos especiales primero
        if self._necesita_regla_cadena(expr):
            return self._aplicar_regla_cadena(expr)
        elif self._necesita_regla_producto(expr):
            return self._aplicar_regla_producto(expr)
        elif self._necesita_regla_cociente(expr):
            return self._aplicar_regla_cociente(expr)

        # Descomposición por términos
        if expr.is_Add:
            display(Markdown("**Paso 1:** Descomponer en términos:"))
            terminos = expr.args
        else:
            terminos = [expr]

        resultados = []
        reglas_usadas = set()

        for term in terminos:
            display(Markdown(f"\n**Término:** `{term}`"))

            if term.is_constant():
                derivada = 0
                regla = "Regla de la constante: d/dx[c] = 0"
            elif term.is_Pow:
                derivada, regla = self._derivar_potencia(term)
            elif term.has(sp.sin, sp.cos, sp.tan):
                derivada, regla = self._derivar_trigonometrica(term)
            elif term.has(sp.exp):
                derivada, regla = self._derivar_exponencial(term)
            elif term.has(sp.log):
                derivada, regla = self._derivar_logaritmica(term)
            else:
                derivada = sp.diff(term, x)
                regla = "Derivada directa"

            display(Markdown(f"**Regla aplicada:** {regla}"))
            display(Markdown(f"**Resultado parcial:** `{derivada}`"))
            resultados.append(derivada)
            reglas_usadas.add(regla)

        # Combinar resultados
        derivada_total = sum(resultados)
        display(Markdown(f"\n**Paso 2:** Combinar derivadas parciales:"))
        display(Math(f"{' + '.join([sp.latex(r) for r in resultados])} = {sp.latex(derivada_total)}"))

        # Simplificar
        derivada_simplificada = sp.simplify(derivada_total)
        if derivada_simplificada != derivada_total:
            display(Markdown("\n**Paso 3:** Simplificar resultado:"))
            display(Math(f"{sp.latex(derivada_total)} = {sp.latex(derivada_simplificada)}"))

        # Mostrar reglas usadas
        display(Markdown("\n**Reglas aplicadas:**"))
        for regla in reglas_usadas:
            display(Markdown(f"- {regla}"))

        display(Markdown(f"\n### 📝 **Derivada final:**"))
        display(Math(f"\\frac{{d}}{{dx}} \\left({sp.latex(expr)}\\right) = {sp.latex(derivada_simplificada)}"))

        return derivada_simplificada

    def _derivar_orden_superior(self, expr, orden):
        """Calcula derivadas de orden superior."""
        x = self.variable
        derivadas = []
        current = expr

        for n in range(1, orden+1):
            display(Markdown(f"\n### 🔄 Calculando {n}° derivada:"))
            derivada = self._derivar_paso_a_paso(current)
            derivadas.append(derivada)
            current = derivada

        display(Markdown(f"\n### 📝 **Derivada de orden {orden}:**"))
        display(Math(f"\\frac{{d^{orden}}}{{dx^{orden}}} \\left({sp.latex(expr)}\\right) = {sp.latex(derivadas[-1])}"))

        return derivadas[-1]

    # ========== REGLAS ESPECIALES ==========
    def _aplicar_regla_cadena(self, expr):
        """Aplica la regla de la cadena para funciones compuestas."""
        x = self.variable
        f, g = self._identificar_funciones_compuestas(expr)

        display(Markdown("**🔗 Aplicando regla de la cadena:**"))
        display(Math(f"\\frac{{d}}{{dx}} f(g(x)) = f'(g(x)) \\cdot g'(x)"))

        df = sp.diff(f, g)
        dg = sp.diff(g, x)
        derivada = df * dg

        display(Markdown("**Paso a paso:**"))
        display(Math(f"f(g) = {sp.latex(f)}, \\quad g(x) = {sp.latex(g)}"))
        display(Math(f"f'(g) = {sp.latex(df)}, \\quad g'(x) = {sp.latex(dg)}"))
        display(Math(f"\\Rightarrow {sp.latex(df)} \\cdot {sp.latex(dg)} = {sp.latex(derivada)}"))

        return derivada

    def _aplicar_regla_producto(self, expr):
        """Aplica la regla del producto para f(x)*g(x)."""
        x = self.variable
        f, g = expr.args

        display(Markdown("**✖ Aplicando regla del producto:**"))
        display(Math(f"\\frac{{d}}{{dx}} [f(x) \\cdot g(x)] = f'(x)g(x) + f(x)g'(x)"))

        df = sp.diff(f, x)
        dg = sp.diff(g, x)
        derivada = df*g + f*dg

        display(Markdown("**Paso a paso:**"))
        display(Math(f"f(x) = {sp.latex(f)}, \\quad g(x) = {sp.latex(g)}"))
        display(Math(f"f'(x) = {sp.latex(df)}, \\quad g'(x) = {sp.latex(dg)}"))
        display(Math(f"\\Rightarrow {sp.latex(df)}{sp.latex(g)} + {sp.latex(f)}{sp.latex(dg)} = {sp.latex(derivada)}"))

        return derivada

    def _aplicar_regla_cociente(self, expr):
        """Aplica la regla del cociente para f(x)/g(x)."""
        x = self.variable
        f, g = expr.as_numer_denom()

        display(Markdown("**➗ Aplicando regla del cociente:**"))
        display(Math(f"\\frac{{d}}{{dx}} \\left[\\frac{{f(x)}}{{g(x)}}\\right] = \\frac{{f'(x)g(x) - f(x)g'(x)}}{{[g(x)]^2}}"))

        df = sp.diff(f, x)
        dg = sp.diff(g, x)
        derivada = (df*g - f*dg) / (g**2)

        display(Markdown("**Paso a paso:**"))
        display(Math(f"f(x) = {sp.latex(f)}, \\quad g(x) = {sp.latex(g)}"))
        display(Math(f"f'(x) = {sp.latex(df)}, \\quad g'(x) = {sp.latex(dg)}"))
        display(Math(f"\\Rightarrow \\frac{{{sp.latex(df)}{sp.latex(g)} - {sp.latex(f)}{sp.latex(dg)}}}{{{sp.latex(g)}^2}} = {sp.latex(derivada)}"))

        return derivada

    # ========== DETECTORES DE REGLAS ==========
    def _necesita_regla_cadena(self, expr):
        """Identifica funciones compuestas para regla de la cadena."""
        return any(arg.has(self.variable) for arg in expr.args if expr.is_Function)

    def _necesita_regla_producto(self, expr):
        """Identifica productos de funciones."""
        return expr.is_Mul and len(expr.args) == 2 and all(arg.has(self.variable) for arg in expr.args)

    def _necesita_regla_cociente(self, expr):
        """Identifica cocientes de funciones."""
        return expr.is_Pow and expr.exp == -1 or expr.is_Mul and any(arg.is_Pow and arg.exp == -1 for arg in expr.args)

    # ========== MÉTODOS PARA TÉRMINOS ESPECÍFICOS ==========
    def _derivar_potencia(self, term):
        """Maneja derivadas de términos con potencias."""
        x = self.variable
        base, exp = term.as_base_exp()

        if base == x:
            derivada = exp * x**(exp - 1)
            regla = f"Regla de la potencia: d/dx[x^{exp}] = {exp}x^{exp-1}"
        elif base.is_Function:
            derivada = exp * base**(exp - 1) * sp.diff(base, x)
            regla = "Regla de la cadena para potencias"
        else:
            derivada = term * sp.log(base) * sp.diff(exp, x) if exp.has(x) else 0
            regla = "Derivada de función exponencial general"

        return derivada, regla

    def _derivar_trigonometrica(self, term):
        """Maneja derivadas de funciones trigonométricas."""
        x = self.variable
        func = next(f for f in (sp.sin, sp.cos, sp.tan) if term.has(f(x)))
        reglas = {
            sp.sin: (sp.cos(x), "d/dx[sin(x)] = cos(x)"),
            sp.cos: (-sp.sin(x), "d/dx[cos(x)] = -sin(x)"),
            sp.tan: (1/sp.cos(x)**2, "d/dx[tan(x)] = sec²(x)")
        }
        derivada, regla = reglas[func]

        if term.args[0] != x:  # Si es función compuesta
            derivada *= sp.diff(term.args[0], x)
            regla = f"Regla de la cadena: {regla}"

        return derivada, regla

    def _derivar_exponencial(self, term):
        """Maneja derivadas de funciones exponenciales."""
        x = self.variable
        base, exp = term.as_base_exp()

        if base == sp.E:
            derivada = term * sp.diff(exp, x) if exp.has(x) else term
            regla = "d/dx[e^u] = e^u * u'" if exp.has(x) else "d/dx[e^x] = e^x"
        else:
            derivada = term * sp.log(base) * sp.diff(exp, x) if exp.has(x) else 0
            regla = "d/dx[a^u] = a^u * ln(a) * u'"

        return derivada, regla

    def _derivar_logaritmica(self, term):
        """Maneja derivadas de funciones logarítmicas."""
        x = self.variable
        if term == sp.log(x):
            return 1/x, "d/dx[ln(x)] = 1/x"
        else:
            derivada = 1/term.args[0] * sp.diff(term.args[0], x)
            return derivada, "Regla de la cadena para logaritmos"

    # ========== MÉTODOS AUXILIARES ==========
    def _parsear_expresion(self, expresion):
        """Convierte una cadena en expresión SymPy."""
        expr_str = expresion.replace("derivar", "").strip()
        return parse_expr(
            expr_str,
            transformations=transformations,
            local_dict={self.variable.name: self.variable}
        )

    def _agregar_al_historial(self, expr, resultado, orden):
        """Guarda en el historial con timestamp."""
        entrada = {
            'fecha': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'expresion': expr,
            'resultado': resultado,
            'orden': orden
        }
        self.historial.append(entrada)

    def mostrar_historial(self):
        """Muestra el historial en formato de tabla."""
        if not self.historial:
            display(Markdown("📜 **El historial está vacío.**"))
            return

        tabla = "| # | Fecha | Expresión | Derivada | Orden |\n|---|---|---|---|---|\n"
        for idx, item in enumerate(self.historial, 1):
            tabla += (
                f"| {idx} | {item['fecha']} | "
                f"`{item['expresion']}` | "
                f"`{item['resultado']}` | {item['orden']}° |\n"
            )
        display(Markdown(f"### 📜 Historial de Derivadas\n{tabla}"))

    def _identificar_funciones_compuestas(self, expr):
        """Identifica f(g(x)) para la regla de la cadena."""
        if expr.is_Function:
            return expr.func, expr.args[0]
        elif expr.is_Pow:
            return expr, expr.base if expr.exp.has(self.variable) else expr.exp
        return expr, self.variable


# Función para facilitar las derivadas
def derivar(expresion, orden=1, variable='x'):
    display(Markdown("---"))
    derivador = DerivadorAvanzado(variable)
    resultado = derivador.calcular_derivada(expresion, orden)
    display(Markdown("---"))
    return resultado

In [5]:
# 1. Derivada simple
derivar("x**3 + 2*x**2 - 5*x + 1")

# 2. Derivada trigonométrica con regla de la cadena
derivar("sin(3*x**2)")

# 3. Derivada de producto
derivar("x*e**x")

# 4. Derivada de cociente
derivar("(x**2 + 1)/(x - 1)")

# 5. Segunda derivada
derivar("log(x**2 + 1)", orden=2)

---

🚀 Derivador Avanzado inicializado (variable: 'x')
Métodos soportados:
- Reglas básicas (potencia, suma, producto, cociente)
- Funciones trigonométricas e hiperbólicas
- Funciones exponenciales y logarítmicas
- Derivación implícita
- Derivadas de orden superior


### 🔍 Derivando (1° orden): `x**3 + 2*x**2 - 5*x + 1`

**Paso 1:** Descomponer en términos:


**Término:** `1`

...

<IPython.core.display.Math object>


### 📝 **Derivada de orden 2:**

<IPython.core.display.Math object>

---

8*x

In [32]:
from IPython.display import display, Markdown, Math
import sympy as sp
from sympy.parsing.sympy_parser import (
    parse_expr,
    standard_transformations,
    implicit_multiplication_application
)
from sympy.integrals.manualintegrate import integral_steps

transformations = (standard_transformations + (implicit_multiplication_application,))

class SuperIntegrador:
    def __init__(self, variable='x'):
        """
        Inicializa el integrador con capacidad para resolver integrales complejas.

        Args:
            variable (str): Variable de integración (por defecto 'x')
        """
        self.variable = sp.Symbol(variable)
        self.historial = []
        print(f"🚀 SuperIntegrador Avanzado inicializado (variable: '{variable}')")
        print("Métodos soportados:")
        print("- Integración por partes")
        print("- Sustitución trigonométrica")
        print("- Fracciones parciales")
        print("- Integrales trigonométricas")
        print("- Sustituciones especiales")

    def calcular_integral(self, expresion, limites=None):
        """
        Calcula integrales indefinidas o definidas con explicación detallada.

        Args:
            expresion (str): Expresión a integrar
            limites (tuple): Límites para integral definida (opcional)

        Returns:
            sympy.Expr: Resultado de la integral
        """
        try:
            expr = self._parsear_expresion(expresion)
            display(Markdown(f"### 🔍 **Integrando:** `{expr}`"))

            # Selección del método de integración
            if self._necesita_sustitucion_trig(expr):
                resultado = self._sustitucion_trigonometrica(expr)
            elif self._necesita_fracciones_parciales(expr):
                resultado = self._fracciones_parciales(expr)
            elif self._necesita_partes(expr):
                resultado = self._integrar_por_partes(expr)
            else:
                resultado = self._integrar_paso_a_paso(expr)

            # Manejo de integral definida
            if limites:
                a, b = map(sp.sympify, limites)
                resultado_definido = resultado.subs(self.variable, b) - resultado.subs(self.variable, a)
                self._mostrar_resultado(expr, resultado, (a, b, resultado_definido))
                self._agregar_al_historial(expr, resultado, f"Definida de {a} a {b}: {resultado_definido}")
                return resultado_definido

            self._mostrar_resultado(expr, resultado)
            self._agregar_al_historial(expr, resultado)
            return resultado

        except Exception as e:
            display(Markdown(f"❌ **Error:** `{str(e)}`"))
            return None

    # ========== MÉTODOS PRINCIPALES DE INTEGRACIÓN ==========
    def _integrar_paso_a_paso(self, expr):
        """Integra mostrando cada paso con reglas aplicadas."""
        x = self.variable
        if expr.is_Add:
            display(Markdown("**Paso 1:** Descomponer en términos:"))
            terminos = expr.args
        else:
            terminos = [expr]

        resultados = []

        for term in terminos:
            display(Markdown(f"\n**Término:** `{term}`"))

            if term.is_constant():
                integral = term * x
                regla = "∫k dx = k·x + C"
            elif term.is_Pow:
                base, exp = term.as_base_exp()
                if exp == -1:
                    integral = sp.log(abs(x))
                    regla = "∫(1/x) dx = ln|x| + C"
                else:
                    integral = (x**(exp + 1)) / (exp + 1)
                    regla = f"∫x^{exp} dx = x^{exp + 1}/{exp + 1} + C"
            elif term.has(sp.sin, sp.cos, sp.tan):
                integral, regla = self._integrar_trigonometrica(term)
            elif term.has(sp.exp):
                integral, regla = self._integrar_exponencial(term)
            elif term.has(sp.log):
                integral, regla = self._integrar_logaritmica(term)
            else:
                integral = sp.integrate(term, x)
                regla = "Integral directa"

            display(Markdown(f"**Regla aplicada:** {regla}"))
            display(Markdown(f"**Resultado parcial:** `{integral}`"))
            resultados.append(integral)

        return sum(resultados)

    def _integrar_por_partes(self, expr):
        """Aplica integración por partes con explicación detallada."""
        x = self.variable

        # Estrategia LIATE para selección de u
        for factor in expr.args:
            if factor.has(sp.log(x)):
                u = factor
                dv = expr / u
                break
            elif factor.has(sp.atan(x), sp.asin(x), sp.acos(x)):
                u = factor
                dv = expr / u
                break
        else:
            u = x
            dv = expr / u

        du = sp.diff(u, x)
        v = sp.integrate(dv, x)
        integral = u * v - sp.integrate(v * du, x)

        display(Markdown(f"""
        **🧩 Integración por partes:**
        - u = {u}
        - dv = {dv} dx
        - du = {du} dx
        - v = ∫{dv} dx = {v}

        **Aplicando fórmula:** ∫u dv = uv - ∫v du
        """))
        display(Markdown(f"**Resultado intermedio:** `{u*v} - ∫{v*du} dx`"))

        return integral

    def _sustitucion_trigonometrica(self, expr):
        """Maneja sustituciones trigonométricas para expresiones con raíces."""
        x = self.variable
        theta = sp.Symbol('θ')

        # Patrones comunes
        a = sp.Wild('a', exclude=[x])
        patterns = [
              sp.sqrt(a**2 - x**2),  # x = a sinθ
              sp.sqrt(a**2 + x**2),   # x = a tanθ
              sp.sqrt(x**2 - a**2)    # x = a secθ
        ]

        for pattern in patterns:
            match = expr.match(pattern)
            if match:
                a_val = match[a]
                if pattern == sp.sqrt(a**2 - x**2):
                    sub = a_val * sp.sin(theta)
                    display(Markdown(f"**Sustitución:** x = {a_val}·sin(θ)"))
                elif pattern == sp.sqrt(a**2 + x**2):
                    sub = a_val * sp.tan(theta)
                    display(Markdown(f"**Sustitución:** x = {a_val}·tan(θ)"))
                else:
                    sub = a_val * sp.sec(theta)
                    display(Markdown(f"**Sustitución:** x = {a_val}·sec(θ)"))

                expr_subs = expr.subs(x, sub) * sp.diff(sub, theta)
                display(Markdown(f"**Expresión sustituida:** `{expr_subs}`"))

                integral = sp.integrate(expr_subs, theta)
                display(Markdown(f"**Integral en términos de θ:** `{integral}`"))

                # Convertir de vuelta a x
                if pattern == sp.sqrt(a**2 - x**2):
                    theta_expr = sp.asin(x/a_val)
                elif pattern == sp.sqrt(a**2 + x**2):
                    theta_expr = sp.atan(x/a_val)
                else:
                    theta_expr = sp.asec(x/a_val)

                resultado = integral.subs(theta, theta_expr)
                display(Markdown(f"**Resultado final:** `{resultado}`"))

                return resultado

        return sp.integrate(expr, x)

    def _fracciones_parciales(self, expr):
        """Aplica descomposición en fracciones parciales."""
        x = self.variable
        expr_apart = sp.apart(expr, x)

        display(Markdown("**🔍 Descomposición en fracciones parciales:**"))
        display(Markdown(f"`{expr} = {expr_apart}`"))

        # Integrar cada término por separado
        if expr_apart.is_Add:
            display(Markdown("\n**Integrando cada término:**"))
            terminos = expr_apart.args
        else:
            terminos = [expr_apart]

        resultados = []
        for term in terminos:
            integral = sp.integrate(term, x)
            display(Markdown(f"- ∫`{term}` dx = `{integral}`"))
            resultados.append(integral)

        return sum(resultados)

    # ========== MÉTODOS AUXILIARES ==========
    def _parsear_expresion(self, expresion):
        """Convierte una cadena en expresión SymPy."""
        try:
            return parse_expr(
                expresion,
                transformations=transformations,
                local_dict={self.variable.name: self.variable}
            )
        except Exception as e:
            raise ValueError(f"Error al parsear la expresión: {str(e)}")

    def _mostrar_resultado(self, expr, resultado, limites=None):
        """Muestra el resultado final con formato profesional."""
        if limites:
            a, b, res_def = limites
            display(Markdown(f"""
            ### 🎉 **Resultado final (integral definida):**
            ```
            ∫_{a}^{b} {expr} dx = {res_def}
            ```
            """))
        else:
            display(Markdown(f"""
            ### 🎉 **Resultado final (integral indefinida):**
            ```
            ∫ {expr} dx = {resultado} + C
            ```
            """))

    def _agregar_al_historial(self, expr, resultado, info_extra=None):
        """Guarda en el historial con timestamp."""
        from datetime import datetime
        entrada = {
            'fecha': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'expresión': expr,
            'resultado': resultado,
            'extra': info_extra or "Indefinida"
        }
        self.historial.append(entrada)

    def mostrar_historial(self):
        """Muestra el historial en formato de tabla."""
        if not self.historial:
            display(Markdown("📜 **El historial está vacío.**"))
            return

        tabla = "| # | Fecha | Expresión | Resultado | Tipo |\n|---|---|---|---|---|\n"
        for idx, item in enumerate(self.historial, 1):
            tabla += (
                f"| {idx} | {item['fecha']} | "
                f"∫`{item['expresión']}` d{self.variable} | "
                f"`{item['resultado']}` | {item['extra']} |\n"
            )
        display(Markdown(f"### 📜 Historial de Integración\n{tabla}"))

    # ========== DETECTORES DE MÉTODOS ==========
    def _necesita_partes(self, expr):
        """Determina si se necesita integración por partes."""
        x = self.variable
        return (expr.is_Mul and any(
            factor.has(sp.log(x), sp.atan(x), sp.asin(x), sp.acos(x))
            for factor in expr.args
        ))

    def _necesita_sustitucion_trig(self, expr):
        """Identifica patrones para sustitución trigonométrica."""
        x = self.variable
        a = sp.Wild('a', exclude=[x])
        patrones = [
            sp.sqrt(a**2 - x**2),
            sp.sqrt(a**2 + x**2),
            sp.sqrt(x**2 - a**2)
        ]
        return any(expr.has(p) for p in patrones)

    def _necesita_fracciones_parciales(self, expr):
        """Reconoce funciones racionales propias."""
        x = self.variable
        return (expr.is_rational_function(x) and
                expr.as_numer_denom()[1].as_poly(x).degree() > 1)

    # ========== MÉTODOS PARA TÉRMINOS ESPECÍFICOS ==========
    def _integrar_trigonometrica(self, term):
        """Integra funciones trigonométricas."""
        x = self.variable
        if term.has(sp.sin(x)):
            return -sp.cos(x), "∫sin(x) dx = -cos(x) + C"
        elif term.has(sp.cos(x)):
            return sp.sin(x), "∫cos(x) dx = sin(x) + C"
        elif term.has(sp.tan(x)):
            return -sp.ln(sp.cos(x)), "∫tan(x) dx = -ln|cos(x)| + C"
        return term, "Integral trigonométrica no identificada"

    def _integrar_exponencial(self, term):
        """Integra funciones exponenciales."""
        x = self.variable
        base, exp = term.as_base_exp()
        if base == sp.E:
            if exp == x:
                return sp.exp(x), "∫e^x dx = e^x + C"
            else:
                return sp.exp(exp)/sp.diff(exp, x), "∫e^u du = e^u + C (u-sub)"
        return term, "Integral exponencial general"

    def _integrar_logaritmica(self, term):
        """Integra funciones logarítmicas."""
        x = self.variable
        if term == sp.log(x):
            return x*sp.log(x) - x, "∫ln(x) dx = x·ln(x) - x + C"
        return term, "Integral logarítmica general"


In [25]:
integ = SuperIntegrador()
integ.calcular_integral("3*x**2 + 4*x")

🔍 SuperIntegrador listo (variable: 'x'). ¡Empecemos!


### 🔍 Integrando: `3*x**2 + 4*x`

**Paso 1:** Descomponer en términos:


**Término:** `3*x**2`


        **Regla aplicada:** Integral directa  
        **Resultado parcial:** `x**3`
        


**Término:** `4*x`


        **Regla aplicada:** Integral directa  
        **Resultado parcial:** `2*x**2`
        


            ### 📝 **Integral indefinida calculada:**
            ```
            ∫ 3*x**2 + 4*x dx = x**3 + 2*x**2 + C
            ```
            

x**3 + 2*x**2

In [6]:
from IPython.display import display, Markdown, Math
import sympy as sp
from sympy.parsing.sympy_parser import (
    parse_expr,
    standard_transformations,
    implicit_multiplication_application
)
from datetime import datetime

transformations = (standard_transformations + (implicit_multiplication_application,))

class CalculadorLimites:
    def __init__(self, variable='x'):
        """
        Calculador avanzado de límites con explicación paso a paso.

        Args:
            variable (str): Variable para el límite (por defecto 'x')
        """
        self.variable = sp.Symbol(variable)
        self.historial = []
        print(f"🧮 Calculador de Límites Avanzado inicializado (variable: '{variable}')")
        print("Capacidades:")
        print("- Límites básicos, laterales y en el infinito")
        print("- Indeterminaciones: 0/0, ∞/∞, ∞-∞, 0*∞, 1^∞, ∞^0, 0^0")
        print("- Técnicas: Factorización, racionalización, L'Hôpital, series de Taylor")
        print("- Funciones: polinómicas, trigonométricas, exponenciales, logarítmicas")

    def calcular_limite(self, expresion, punto, direccion='+'):
        """
        Calcula límites con explicación detallada.

        Args:
            expresion (str): Expresión a evaluar
            punto (str): Punto hacia donde tiende el límite (puede ser 'oo' para infinito)
            direccion (str): '+' para límite por derecha, '-' por izquierda

        Returns:
            sympy.Expr: Resultado del límite
        """
        try:
            expr = self._parsear_expresion(expresion)
            punto_eval = sp.oo if str(punto).lower() in ['oo', 'inf', 'infinito'] else parse_expr(str(punto))

            display(Markdown(f"### 🔍 Calculando límite: {'lím' if direccion == '+' else 'lím⁻'}"))
            display(Math(f"\\lim_{{{self.variable} \\to {sp.latex(punto_eval)}{'^+' if direccion == '+' else '^-'}}} {sp.latex(expr)}"))

            # Paso 1: Evaluación directa
            valor_directo = self._evaluar_directamente(expr, punto_eval, direccion)
            if valor_directo is not None:
                self._mostrar_resultado(expr, punto_eval, valor_directo, direccion)
                self._agregar_al_historial(expr, punto_eval, valor_directo, direccion)
                return valor_directo

            # Paso 2: Identificar tipo de indeterminación
            indeterminacion = self._identificar_indeterminacion(expr, punto_eval, direccion)
            display(Markdown(f"**Indeterminación encontrada:** `{indeterminacion}`"))

            # Paso 3: Aplicar técnica adecuada
            if indeterminacion == '0/0':
                resultado = self._resolver_0_sobre_0(expr, punto_eval, direccion)
            elif indeterminacion == 'oo/oo':
                resultado = self._resolver_inf_sobre_inf(expr, punto_eval, direccion)
            elif indeterminacion == '0*oo':
                resultado = self._resolver_0_por_inf(expr, punto_eval, direccion)
            elif indeterminacion == 'oo-oo':
                resultado = self._resolver_inf_menos_inf(expr, punto_eval, direccion)
            elif indeterminacion in ['1^oo', '0^0', 'oo^0']:
                resultado = self._resolver_potencias_indeterminadas(expr, punto_eval, direccion)
            else:
                resultado = sp.limit(expr, self.variable, punto_eval, dir=direccion)

            self._mostrar_resultado(expr, punto_eval, resultado, direccion)
            self._agregar_al_historial(expr, punto_eval, resultado, direccion)
            return resultado

        except Exception as e:
            display(Markdown(f"❌ **Error:** `{str(e)}`"))
            return None

    # ========== MÉTODOS PRINCIPALES ==========
    def _evaluar_directamente(self, expr, punto, direccion):
        """Intenta evaluación directa del límite."""
        try:
            valor = expr.subs(self.variable, punto)
            if not valor.has(sp.nan, sp.zoo, sp.oo, -sp.oo):
                display(Markdown("**Evaluación directa:** Sustitución no produce indeterminación"))
                return valor

            # Verificar límites laterales para discontinuidades
            if str(punto) != 'oo':
                limite_izq = sp.limit(expr, self.variable, punto, dir='-')
                limite_der = sp.limit(expr, self.variable, punto, dir='+')

                if limite_izq == limite_der:
                    display(Markdown("**Límite existe:** Ambos laterales coinciden"))
                    return limite_izq
                else:
                    display(Markdown(f"**Límite no existe:** Laterales diferentes ({limite_izq} ≠ {limite_der})"))
                    return None

            return None
        except:
            return None

    def _identificar_indeterminacion(self, expr, punto, direccion):
        """Determina el tipo de indeterminación."""
        if str(punto) == 'oo':
            expr_rewritten = expr.subs(self.variable, 1/self.variable)
            punto = 0
        else:
            expr_rewritten = expr

        try:
            num, den = expr_rewritten.as_numer_denom()
            val_num = num.subs(self.variable, punto)
            val_den = den.subs(self.variable, punto)

            if val_num == 0 and val_den == 0:
                return '0/0'
            elif abs(val_num) == sp.oo and abs(val_den) == sp.oo:
                return 'oo/oo'
            elif (val_num == 0 and abs(val_den) == sp.oo) or (abs(val_num) == sp.oo and val_den == 0):
                return '0*oo'

        except:
            pass

        # Para diferencias
        if expr_rewritten.is_Add:
            terms = expr_rewritten.args
            vals = [t.subs(self.variable, punto) for t in terms]
            if any(v == sp.oo for v in vals) and any(v == -sp.oo for v in vals):
                return 'oo-oo'

        # Para potencias
        if expr_rewritten.is_Pow:
            base, exp = expr_rewritten.as_base_exp()
            val_base = base.subs(self.variable, punto)
            val_exp = exp.subs(self.variable, punto)

            if val_base == 1 and abs(val_exp) == sp.oo:
                return '1^oo'
            elif val_base == 0 and val_exp == 0:
                return '0^0'
            elif abs(val_base) == sp.oo and val_exp == 0:
                return 'oo^0'

        return 'Indeterminación desconocida'

    # ========== TÉCNICAS PARA INDETERMINACIONES ==========
    def _resolver_0_sobre_0(self, expr, punto, direccion):
        """Resuelve indeterminaciones 0/0."""
        display(Markdown("**Aplicando técnicas para 0/0:**"))

        # 1. Factorización
        expr_fact = sp.factor(expr)
        if expr_fact != expr:
            display(Markdown("- **Factorización:**"))
            display(Math(f"{sp.latex(expr)} = {sp.latex(expr_fact)}"))
            resultado = self._evaluar_directamente(expr_fact, punto, direccion)
            if resultado is not None:
                return resultado

        # 2. Racionalización
        if any(sp.sqrt(f) in expr.args for f in expr.args if f.is_Pow):
            display(Markdown("- **Racionalización:**"))
            conjugado = self._encontrar_conjugado(expr)
            if conjugado:
                expr_rac = expr * conjugado / conjugado
                expr_rac = sp.simplify(expr_rac)
                display(Math(f"\\frac{{{sp.latex(expr)} \\cdot {sp.latex(conjugado)}}}{{{sp.latex(conjugado)}}} = {sp.latex(expr_rac)}"))
                resultado = self._evaluar_directamente(expr_rac, punto, direccion)
                if resultado is not None:
                    return resultado

        # 3. Regla de L'Hôpital
        display(Markdown("- **Aplicando Regla de L'Hôpital:**"))
        num, den = expr.as_numer_denom()
        dnum = sp.diff(num, self.variable)
        dden = sp.diff(den, self.variable)

        display(Math(f"\\lim \\frac{{f(x)}}{{g(x)}} = \\lim \\frac{{f'(x)}}{{g'(x)}}"))
        display(Math(f"\\frac{{d/dx[{sp.latex(num)}]}}{{d/dx[{sp.latex(den)}]}} = \\frac{{{sp.latex(dnum)}}}{{{sp.latex(dden)}}}"))

        nuevo_lim = dnum / dden
        resultado = self._evaluar_directamente(nuevo_lim, punto, direccion)
        if resultado is not None:
            return resultado
        else:
            return sp.limit(expr, self.variable, punto, dir=direccion)

    def _resolver_inf_sobre_inf(self, expr, punto, direccion):
        """Resuelve indeterminaciones ∞/∞."""
        display(Markdown("**Aplicando técnicas para ∞/∞:**"))

        # 1. Dividir por término de mayor grado
        if expr.is_rational_function(self.variable):
            display(Markdown("- **Dividir por la mayor potencia:**"))
            grado_num = sp.degree(sp.numer(expr), self.variable)
            grado_den = sp.degree(sp.denom(expr), self.variable)

            if grado_num > grado_den:
                resultado = sp.oo if sp.Limit(sp.numer(expr)/sp.denom(expr), self.variable, sp.oo).doit() > 0 else -sp.oo
            elif grado_num < grado_den:
                resultado = 0
            else:
                coef_num = sp.LC(sp.numer(expr), self.variable)
                coef_den = sp.LC(sp.denom(expr), self.variable)
                resultado = coef_num / coef_den

            display(Math(f"\\frac{{{sp.latex(sp.numer(expr))}}}{{{sp.latex(sp.denom(expr))}}} \\Rightarrow {sp.latex(resultado)}"))
            return resultado

        # 2. Regla de L'Hôpital
        display(Markdown("- **Aplicando Regla de L'Hôpital:**"))
        num, den = expr.as_numer_denom()
        dnum = sp.diff(num, self.variable)
        dden = sp.diff(den, self.variable)
        return sp.limit(dnum/dden, self.variable, punto, dir=direccion)

    def _resolver_0_por_inf(self, expr, punto, direccion):
        """Resuelve indeterminaciones 0*∞."""
        display(Markdown("**Aplicando técnicas para 0*∞:**"))
        display(Markdown("- Convertir a forma 0/0 o ∞/∞:"))

        # Convertir a 0/0
        term1, term2 = expr.args if len(expr.args) == 2 else (expr.args[0], sp.Mul(*expr.args[1:]))

        if term1.subs(self.variable, punto) == 0:
            nuevo_expr = term1 / (1/term2)
        else:
            nuevo_expr = term2 / (1/term1)

        display(Math(f"{sp.latex(expr)} = \\frac{{{sp.latex(term1)}}}{{{sp.latex(1/term2)}}}"))
        return self.calcular_limite(nuevo_expr, punto, direccion)

    def _resolver_inf_menos_inf(self, expr, punto, direccion):
        """Resuelve indeterminaciones ∞-∞."""
        display(Markdown("**Aplicando técnicas para ∞-∞:**"))

        # 1. Factor común
        if expr.is_Add:
            display(Markdown("- **Factor común:**"))
            factor = sp.factor(expr)
            if factor != expr:
                display(Math(f"{sp.latex(expr)} = {sp.latex(factor)}"))
                return self.calcular_limite(factor, punto, direccion)

        # 2. Combinar fracciones
        display(Markdown("- **Combinar fracciones:**"))
        expr_comb = sp.together(expr)
        if expr_comb != expr:
            display(Math(f"{sp.latex(expr)} = {sp.latex(expr_comb)}"))
            return self.calcular_limite(expr_comb, punto, direccion)

        return sp.limit(expr, self.variable, punto, dir=direccion)

    def _resolver_potencias_indeterminadas(self, expr, punto, direccion):
        """Resuelve indeterminaciones 1^∞, 0^0, ∞^0."""
        display(Markdown("**Aplicando técnicas para formas exponenciales:**"))

        base, exp = expr.as_base_exp()
        display(Markdown("- Aplicando logaritmo natural:"))
        nuevo_expr = exp * sp.log(base)
        display(Math(f"\\lim {sp.latex(expr)} = e^{{\\lim {sp.latex(nuevo_expr)}}}"))

        limite_exp = self.calcular_limite(nuevo_expr, punto, direccion)
        if limite_exp is not None:
            resultado = sp.exp(limite_exp)
            display(Math(f"= e^{{{sp.latex(limite_exp)}}} = {sp.latex(resultado)}"))
            return resultado

        return sp.limit(expr, self.variable, punto, dir=direccion)

    # ========== MÉTODOS AUXILIARES ==========
    def _parsear_expresion(self, expresion):
        """Convierte una cadena en expresión SymPy."""
        expr_str = expresion.replace("limite", "").strip()
        return parse_expr(
            expr_str,
            transformations=transformations,
            local_dict={self.variable.name: self.variable}
        )

    def _encontrar_conjugado(self, expr):
        """Encuentra el conjugado para racionalizar."""
        for term in expr.args:
            if term.is_Pow and term.exp == sp.S(1)/2:
                return term.subs(term.args[0], term.args[0])
            elif term.is_Mul:
                for f in term.args:
                    if f.is_Pow and f.exp == sp.S(1)/2:
                        return f.subs(f.args[0], f.args[0])
        return None

    def _mostrar_resultado(self, expr, punto, resultado, direccion):
        """Muestra el resultado final."""
        display(Markdown(f"### 📝 **Resultado del límite:**"))
        if str(punto) == 'oo':
            punto_str = "\\infty"
        elif str(punto) == '-oo':
            punto_str = "-\\infty"
        else:
            punto_str = str(punto)

        display(Math(
            f"\\lim_{{{self.variable} \\to {punto_str}{'^+' if direccion == '+' else '^-'}}} "
            f"{sp.latex(expr)} = {sp.latex(resultado)}"
        ))

    def _agregar_al_historial(self, expr, punto, resultado, direccion):
        """Guarda en el historial con timestamp."""
        entrada = {
            'fecha': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'expresion': expr,
            'punto': punto,
            'resultado': resultado,
            'direccion': direccion
        }
        self.historial.append(entrada)

    def mostrar_historial(self):
        """Muestra el historial en formato de tabla."""
        if not self.historial:
            display(Markdown("📜 **El historial está vacío.**"))
            return

        tabla = "| # | Fecha | Expresión | Punto | Dirección | Resultado |\n|---|---|---|---|---|---|\n"
        for idx, item in enumerate(self.historial, 1):
            tabla += (
                f"| {idx} | {item['fecha']} | "
                f"`{item['expresion']}` | "
                f"{item['punto']} | "
                f"{item['direccion']} | "
                f"{item['resultado']} |\n"
            )
        display(Markdown(f"### 📜 Historial de Límites\n{tabla}"))


# Función para facilitar el cálculo de límites
def limite(expresion, punto, direccion='+', variable='x'):
    display(Markdown("---"))
    calculador = CalculadorLimites(variable)
    resultado = calculador.calcular_limite(expresion, punto, direccion)
    display(Markdown("---"))
    return resultado


    # Mostrar historial
    calculador = CalculadorLimites()
    calculador.mostrar_historial()

In [8]:
# 1. Límite básico que lleva a 0/0
limite("(x**2 - 9)/(x - 3)", 3)

# 2. Límite en el infinito (∞/∞)
limite("(2x**3 + x - 1)/(5x**3 - x**2 + 4)", "oo")

# 3. Límite trigonométrico importante
limite("(1 - cos(x))/x**2", 0)

# 4. Límite lateral
limite("1/(x - 2)", 2, direccion='+')

# 5. Indeterminación 1^∞ (límite fundamental)
limite("(1 + 3/x)**(2x)", "oo")

# 6. Indeterminación ∞ - ∞
limite("sqrt(x**2 + 1) - x", "oo")

---

🧮 Calculador de Límites Avanzado inicializado (variable: 'x')
Capacidades:
- Límites básicos, laterales y en el infinito
- Indeterminaciones: 0/0, ∞/∞, ∞-∞, 0*∞, 1^∞, ∞^0, 0^0
- Técnicas: Factorización, racionalización, L'Hôpital, series de Taylor
- Funciones: polinómicas, trigonométricas, exponenciales, logarítmicas


### 🔍 Calculando límite: lím

<IPython.core.display.Math object>

**Límite existe:** Ambos laterales coinciden

...

**Indeterminación encontrada:** `Indeterminación desconocida`

### 📝 **Resultado del límite:**

<IPython.core.display.Math object>

---

0